## Code for all_countries_2016.csv 

In [0]:
!pip install quandl

In [0]:
import quandl
import pandas as pd
import numpy as np
import pandas as pd
from google.colab import drive, files
import sys
import time
import os
import shutil

#quandl.ApiConfig.api_key = "GxX1kJbkSAeX5YsK2tsS"
#quandl.ApiConfig.api_key = "QJcFTfnEZXrKsKHETczD"

In [0]:
#mount google drive files 
drive.mount('/content/drive')

#copy csv's to cloud
!cp drive/My\ Drive/CDS_project/country_codes drive/My\ Drive/CDS_project/Cpi18.csv drive/My\ Drive/CDS_project/wwdi_indicators .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#countries
countries = pd.read_csv("country_codes", sep="|",header=0)
countries.columns=['country','country_code']

#indicators 
indicators= pd.read_csv("wwdi_indicators",sep="|",header=0)
indicators.columns=['indicator','indicator_code']

c_code = countries['country_code']
i_code = indicators['indicator_code']
i_name = indicators['indicator']

i_code_arr = list(i_code)

### Pull all data in from Quandl (parallelised)

In [0]:
for i in l:
  print(i)
  os.mkdir(f'drive/My Drive/CDS_project/data/{i}')
  for j in i_code:
    try:
      code = f'{i}_{j}'
      this_df = quandl.get(f'WWDI/{i}_{j}')

      if not this_df.empty:
        this_df.columns = [f'{code}']
        this_df.to_csv(f'{code}.csv')
        shutil.move(f'{code}.csv', f'drive/My Drive/CDS_project/data/{i}')

      else:
        print(f'found empty df for code {code}')
        pd.DataFrame().to_csv(f'{code}.csv')
        shutil.move(f'{code}.csv', f'drive/My Drive/CDS_project/data/{i}')

    except quandl.errors.quandl_error.NotFoundError:
      pd.DataFrame().to_csv(f'{code}.csv')
      shutil.move(f'{code}.csv', f'drive/My Drive/CDS_project/data/{i}')

  print('sleeping')
  time.sleep(60*5)


### Build dataframe for all_countries_2016.csv

In [0]:
countries = os.listdir('drive/My Drive/CDS_project/data')[45:90]
counter = 0
df = pd.DataFrame()
for i in countries:
  print(i)
  with os.scandir(f'drive/My Drive/CDS_project/data/{i}') as dir:
    row = pd.DataFrame(columns=i_code_arr) #current row
    for f in dir:
      f_size = f.stat().st_size
      this_i_code = f.name[4:][:-4]
      #print(f.name)
      #print(f'size is {f_size}')
      if f_size > 5: #open csv and find entry
        this_df = pd.read_csv(f, header=0, index_col=0)
        this_df.index = pd.to_datetime(this_df.index)

        if 2016 in this_df.index.year: #entry present
          idx = list(this_df.index.year).index(2016)
          row[f'{this_i_code}'] = [this_df.iloc[idx][0]]

        else: #entry not present
          last = this_df.index.year[-1]
          idx = list(this_df.index.year).index(last)
          row[f'{this_i_code}'] = [this_df.iloc[idx][0]]
          counter += 1
          
      else: #add nan for this indicator
        row[f'{this_i_code}'] = [np.nan]

    df = df.append(row)

df.to_csv('2.csv', index=False)
shutil.copy('2.csv', 'drive/My Drive/CDS_project')